In [1]:
from omegaconf import OmegaConf

config = OmegaConf.load("config.yaml")
config

{'backbone_path': 'siglip_checkpoints', 'training': True, 'val_dataset': False, 'temperature': 0.02, 'img_aug': True, 'cmt_depth': 4, 'lr_factor': 5.0, 'MLM': True, 'loss_names': ['sdm', 'id', 'mlm', 'itc', 'cmpm'], 'mlm_loss_weight': 1.0, 'id_loss_weight': 1.0, 'img_size': [384, 128], 'interpolate_pos_encoding': True, 'stride_size': 16, 'text_length': 64, 'vocab_size': 250001, 'optimizer': 'AdamW', 'lr': 1e-05, 'bias_lr_factor': 2.0, 'momentum': 0.9, 'weight_decay': 4e-05, 'weight_decay_bias': 0.0, 'alpha': 0.9, 'beta': 0.999, 'num_epochs': 60, 'milestones': [20, 50], 'gamma': 0.1, 'warmup_factor': 0.1, 'warmup_epochs': 5, 'warmup_method': 'linear', 'lrscheduler': 'cosine', 'target_lr': 0, 'power': 0.9, 'dataset_name': 'VN3K', 'sampler': 'random', 'num_intance': 4, 'root_dir': '.', 'batch_size': 128, 'test_batch_size': 512, 'num_workers': 4}

In [2]:
config.batch_size = 1

In [3]:
from lightning_data import IRRADataModule

dm = IRRADataModule(config)

/home/phongtnh/miniconda3/envs/person_search/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-06-10 21:55:07.193 | INFO     | datasets.vn3k:__init__:29 - => VN3K Images and Captions are loaded
2024-06-10 21:55:07.193 | INFO     | datasets.bases:show_dataset_info:28 - VN3K Dataset statistics:
2024-06-10 21:55:07.194 | INFO     | datasets.bases:show_dataset_info:35 - 
+--------+------+--------+----------+
| subset | ids  | images | captions |
+--------+------+--------+----------+
| train  | 2000 |  4302  |  17208   |
|  test  | 1000 |  2000  |   8000   |
|  val   |  0   |   0    |    0     |
+--------+------+--------+----------+


VN3K


In [5]:
dm.setup()

In [6]:
train = dm.train_set

In [8]:
print("Image shape:", train[0]['images'].shape)
print("Caption shape:", train[0]['caption_input']['input_ids'].shape)

Image shape: torch.Size([3, 384, 128])
Caption shape: torch.Size([1, 34])


In [5]:
train_loader = dm.train_dataloader()
val_loader = dm.val_dataloader()
test_loader = dm.test_dataloader()

2024-06-10 21:43:39.118 | INFO     | lightning_data:train_dataloader:109 - using random sampler


In [6]:
train_loader.batch_size

1

In [7]:
config.num_epochs

60

In [9]:
import lightning as L
from lighning_models import LitIRRA

model = LitIRRA(config, img_loader=test_loader[0], text_loader=test_loader[1], num_classes=dm.num_classes)

2024-06-10 21:43:57.454 | INFO     | model.build:__init__:19 - Training Model with ['sdm', 'id', 'mlm', 'itc', 'cmpm'] tasks
2024-06-10 21:43:59.280 | INFO     | model.build:__init__:28 - Embedding Dimension: 768


In [12]:
from loguru import logger
trainer = L.Trainer(fast_dev_run=2, precision="bf16-mixed", logger=logger, num_sanity_val_steps=2)

Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Running in `fast_dev_run` mode: will run the requested loop using 2 batch(es). Logging and checkpointing is suppressed.


In [13]:
trainer.fit(model, train_dataloaders=train_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type | Params
-------------------------------
0 | model | IRRA | 595 M 
-------------------------------
595 M     Trainable params
0         Non-trainable params
595 M     Total params
2,382.905 Total estimated model params size (MB)


Using 5.0 times learning rate for random init module 
Epoch 0: 100%|██████████| 2/2 [00:01<00:00,  1.65it/s, temperature_step=0.020, itc_loss_step=0.000, sdm_loss_step=0.000, cmpm_loss_step=0.000, id_loss_step=7.610, img_acc_step=0.000, txt_acc_step=0.000, mlm_loss_step=12.40, mlm_acc_step=0.000, temperature_epoch=0.020, itc_loss_epoch=0.000, sdm_loss_epoch=0.000, cmpm_loss_epoch=0.000, id_loss_epoch=7.590, img_acc_epoch=0.000, txt_acc_epoch=0.000, mlm_loss_epoch=12.40, mlm_acc_epoch=0.000]

`Trainer.fit` stopped: `max_steps=2` reached.


Epoch 0: 100%|██████████| 2/2 [00:01<00:00,  1.65it/s, temperature_step=0.020, itc_loss_step=0.000, sdm_loss_step=0.000, cmpm_loss_step=0.000, id_loss_step=7.610, img_acc_step=0.000, txt_acc_step=0.000, mlm_loss_step=12.40, mlm_acc_step=0.000, temperature_epoch=0.020, itc_loss_epoch=0.000, sdm_loss_epoch=0.000, cmpm_loss_epoch=0.000, id_loss_epoch=7.590, img_acc_epoch=0.000, txt_acc_epoch=0.000, mlm_loss_epoch=12.40, mlm_acc_epoch=0.000]


In [ ]:
def collate(batch):
    # Initialize dictionary to hold batched data
    batched_data = {}

    # Get all keys from the first item in the batch to identify field names
    keys = batch[0].keys()

    for key in keys:
        if isinstance(batch[0][key], dict):
            # If the field is a nested dictionary, process each sub-field
            sub_keys = batch[0][key].keys()
            batched_data[key] = {}
            for sub_key in sub_keys:
                sub_values = [item[key][sub_key] for item in batch]
                if isinstance(sub_values[0], torch.Tensor):
                    # Concatenate tensors
                    batched_data[key][sub_key] = torch.cat(sub_values, dim=0)
                else:
                    # Assume list of primitive types and convert to tensor
                    batched_data[key][sub_key] = torch.tensor(sub_values, dtype=torch.int64)
        else:
            # If the field is not a nested dictionary, process directly
            values = [item[key] for item in batch]
            if isinstance(values[0], torch.Tensor):
                # Stack tensors if they are of the same size
                batched_data[key] = torch.stack(values)
            else:
                # Assume list of primitive types and convert to tensor
                batched_data[key] = torch.tensor(values, dtype=torch.int64)

    return batched_data

train = dm.train_set

In [ ]:
train[2]

{'pids': 0,
 'image_ids': 0,
 'images': tensor([[[ 0.0617,  0.0763,  0.0471,  ..., -1.7923, -1.7923, -1.7923],
          [ 0.1055,  0.1055,  0.0617,  ..., -1.7923, -1.7923, -1.7923],
          [ 0.0325,  0.0471,  0.0325,  ..., -1.7923, -1.7923, -1.7923],
          ...,
          [-1.7923, -1.7923, -1.7923,  ..., -1.7923, -1.7923, -1.7923],
          [-1.7923, -1.7923, -1.7923,  ..., -1.7923, -1.7923, -1.7923],
          [-1.7923, -1.7923, -1.7923,  ..., -1.7923, -1.7923, -1.7923]],
 
         [[-0.6415, -0.6565, -0.6865,  ..., -1.7521, -1.7521, -1.7521],
          [-0.5965, -0.6265, -0.6715,  ..., -1.7521, -1.7521, -1.7521],
          [-0.6565, -0.6715, -0.7016,  ..., -1.7521, -1.7521, -1.7521],
          ...,
          [-1.7521, -1.7521, -1.7521,  ..., -1.7521, -1.7521, -1.7521],
          [-1.7521, -1.7521, -1.7521,  ..., -1.7521, -1.7521, -1.7521],
          [-1.7521, -1.7521, -1.7521,  ..., -1.7521, -1.7521, -1.7521]],
 
         [[-0.3853, -0.3711, -0.3995,  ..., -1.4802, -1.4802,